# Merging Data to Understand the Relationship between Drug Legalization and Violent Crime
### Rachel Richards and Sydney Donati-Leach

## Pre-Legalization Analysis

### 1.) Download and import the file ca_arrests_2009.csv

In [1]:
import pandas as pd
arrests_2009 = pd.read_csv('https://raw.githubusercontent.com/nickeubank/practicaldatascience/master/Example_Data/ca/ca_arrests_2009.csv')

### 2.) Get a feel for the data

In [2]:
arrests_2009.count()

Unnamed: 0    58
COUNTY        58
VIOLENT       58
PROPERTY      58
F_DRUGOFF     58
F_SEXOFF      58
F_ALLOTHER    58
F_TOTAL       58
M_TOTAL       58
S_TOTAL       58
dtype: int64

### 3.) Import nhgis_county_populations.csv

In [3]:
pop_2009 = pd.read_csv('https://raw.githubusercontent.com/nickeubank/practicaldatascience/master/Example_Data/ca/nhgis_county_populations.csv')

### 4.) Figure out how this data relates to your 2009 arrest data.

In [4]:
ca_pop = pop_2009[pop_2009.STATE == "California"] 
ca_pop_2009 = ca_pop[ca_pop.YEAR == "2005-2009"]
ca_pop_2009.count()

Unnamed: 0          58
YEAR                58
STATE               58
COUNTY              58
total_population    58
dtype: int64

### 5.) Merge the two datasets.

In [5]:
merged_data_2009 = pd.merge(arrests_2009, ca_pop_2009)

## Checking Your Merges

### 6.) 
- What variable do you think will be consistent across these two datasets you can use for merging?
    - We think that "COUNTY" will be the column our datasets will merge on.
- Do you think there will be exactly 1 observation for each value in your arrest data?
    - Yes, there should only be one value for each person. But multiple people can be in each county.
- Do you think there will be exactly 1 observation for each value in your population data?
    - Yes, there should only be one value for each county. However, not every county will be represented in the arrest data.

### Repeat the merge you conducted above, but this time use the validate to make sure your assumptions about the data were correct.

In [6]:
merged_data_2009 = pd.merge(arrests_2009, ca_pop_2009, on="COUNTY", validate = "one_to_one", how="left")

### 7.) Create an indicator.

In [7]:
merged_data_2009 = pd.merge(arrests_2009, ca_pop_2009, on="COUNTY", validate = "one_to_one", how="left", indicator=True)

### 8.) Repeat your previous merge using both the validate keyword and the indicator keyword with how='outer'.

In [8]:
merged_data_2009 = pd.merge(arrests_2009, ca_pop_2009, on="COUNTY", validate = "one_to_one", how="outer", indicator=True)

### 9.)  Figure out why all counties in our arrest data did not merge with population data. Fix the data so that they all merge to population data.
At first not all of the counties merged into the dataset because there are two counties that are in the arrest dataset that are not in the population dataset. Thus, when an inner merge is performed the new dataset did not include those two counties. They are included when a left merge is used, which is shown below.

In [9]:
merged_data_2009 = pd.merge(arrests_2009, ca_pop_2009, on="COUNTY", validate = "one_to_one", how="left", indicator=True)

## Comparing Arrest Rates

### 10.) For each county, create a new variable called 'violent_arrest_rate_2009' that is the number of violent arrests for 2009 divided by the population of the county from 2005-2009, and an analogous variable for drug offenses (F_DRUGOFF).

In [10]:
merged_data_2009["violent_arrest_rate_2009"] = merged_data_2009["VIOLENT"] / merged_data_2009["total_population"]
merged_data_2009["drug_arrest_rate_2009"] = merged_data_2009["F_DRUGOFF"] / merged_data_2009["total_population"]

### 11.) Make a scatter plot that shows the relationship between each county’s violent arrest rate and it’s drug arrest rate.

In [11]:
import altair as alt

c = alt.Chart(merged_data_2009).mark_point().encode(x= alt.X(f"violent_arrest_rate_2009", scale=alt.Scale(zero=False)), y= f"drug_arrest_rate_2009")
c.display()

alt.Chart(...)

From the chart we can see that there is a slight trend between violent and drug arrest rates. As the violent arrest rate increase so does the drug arrest rate.

## Comparing with 2018 Arrests

### 12.) Now use the dataset to find the rates for 2018.

In [12]:
arrests_2018 = pd.read_csv('https://raw.githubusercontent.com/nickeubank/practicaldatascience/master/Example_Data/ca/ca_arrests_2018.csv')
ca_pop_2018 = ca_pop[ca_pop.YEAR == "2013-2017"]
merged_data_2018 = pd.merge(arrests_2018, ca_pop_2018, on="COUNTY", validate = "one_to_one", how="left")
merged_data_2018["violent_arrest_rate_2018"] = merged_data_2018["VIOLENT"] / merged_data_2018["total_population"]
merged_data_2018["drug_arrest_rate_2018"] = merged_data_2018["F_DRUGOFF"] / merged_data_2018["total_population"]

### 13.) Merge the two county-level datasets so you have one row for every county, and variables for violent arrest rates in 2018, violent arrest rates in 2009, felony drug arrest rates in 2018, and felony drug arrest rates in 2009.

In [13]:
merged_years= pd.merge(merged_data_2009, merged_data_2018, on="COUNTY", validate = "one_to_one", how="left")

### 14.) Did drug arrests go down from 2009 to 2018?

In [14]:
avg_drug_2009 = merged_years["drug_arrest_rate_2009"].mean()
avg_drug_2018 = merged_years["drug_arrest_rate_2018"].mean()
print("The average percent of drug arrests in 2009 was {:.2f}%.".format(avg_drug_2009 * 100))
print("The average percent of drug arrests in 2018 was {:.2f}%.".format(avg_drug_2018 * 100))

The average percent of drug arrests in 2009 was 0.32%.
The average percent of drug arrests in 2018 was 0.10%.


This indicates that drug arrests went down from 2009 to 2018.

### 15.) Did the average violent arrest rate decrease? By how much?

In [15]:
avg_violent_2009 = merged_years["violent_arrest_rate_2009"].mean()
avg_violent_2018 = merged_years["violent_arrest_rate_2018"].mean()

print("The average percent of violent arrests in 2009 was {:.2f}%.".format(avg_violent_2009 * 100))
print("The average percent of violent arrests in 2018 was {:.2f}%.".format(avg_violent_2018 * 100))

The average percent of violent arrests in 2009 was 0.37%.
The average percent of violent arrests in 2018 was 0.33%.


The average violent arrests decreased by 0.04% from 2009 to 2018.

### 16.) Split the sample into two groups: high drug arrests in 2009, and low drug arrests in 2009 (cut the sample at the average drug arrest rate in 2009). Did violent crime fall more from 2009 to 2018 in the counties that had lots of drug arrests in 2009 (where legalization likely had more of an effect) than in counties with fewer drug arrests in 2009 (where legalization likely mattered less)?

In [16]:
low_drug_arrest = merged_years[merged_years["drug_arrest_rate_2009"] < 0.0032163867]
high_drug_arrest = merged_years[merged_years["drug_arrest_rate_2009"] >= 0.0032163867]

avg_violent_lowdrug_2009 = low_drug_arrest["violent_arrest_rate_2009"].mean()
avg_violent_highdrug_2009 = high_drug_arrest["violent_arrest_rate_2009"].mean()

diff_low = avg_violent_2018 - avg_violent_lowdrug_2009
diff_high = avg_violent_2018 - avg_violent_highdrug_2009

print("The change in violent crime rate for counties with lots of drug arrests in 2009 was {:.3f}%.".format(diff_low * 100))
print("The change in violent crime rate for counties with few drug arrests in 2009 was {:.3f}%.".format(diff_high * 100))

#(the change in violent crime rate for counties with lots of drug arrests in 2009) - (the change in violent crime rate for counties with few drug arrests in 2009)
diff_of_diff = diff_high - diff_low

print("Counties with high drug arrests had violent crime rates decrease by {:.3f}% more than counties with low drug arrests.".format(diff_of_diff * 100))



The change in violent crime rate for counties with lots of drug arrests in 2009 was 0.003%.
The change in violent crime rate for counties with few drug arrests in 2009 was -0.083%.
Counties with high drug arrests had violent crime rates decrease by -0.086% more than counties with low drug arrests.


### 17.) Calculate the proportionate decline in violent arrest rates in counties more impacted by drug legalization as compared to counties less impacted by drug legalization.

In [17]:
avg_lowdrug_2009 = low_drug_arrest["drug_arrest_rate_2009"].mean()

percent_change_low = ((avg_violent_2018 - avg_violent_lowdrug_2009)/ avg_violent_lowdrug_2009) * 100
percent_change_high = ((avg_violent_2018 - avg_violent_highdrug_2009)/ avg_violent_highdrug_2009) * 100

print("The percentage change in violent crime rate for counties with few drug arrests in 2009 was {:.3f}%.".format(percent_change_low))
print("The percentage change in violent crime rate for counties with lots of drug arrests in 2009 was {:.3f}%.".format(percent_change_high))

#(the percentage change in violent crime rate for counties with lots of drug arrests in 2009) - (the percentage change in violent crime rate for counties with few drug arrests in 2009)
proportionate_diff = percent_change_high - percent_change_low

print("The proportional difference between percent change in violent crime for counties with lots of drug arrests versus counties with few drug arrests is {:.3f}%. This shows that counties with high drug arrests had a larger proportionate decline.".format(proportionate_diff))

The percentage change in violent crime rate for counties with few drug arrests in 2009 was 0.916%.
The percentage change in violent crime rate for counties with lots of drug arrests in 2009 was -19.901%.
The proportional difference between percent change in violent crime for counties with lots of drug arrests versus counties with few drug arrests is -20.816%. This shows that counties with high drug arrests had a larger proportionate decline.
